#### Name: {Broker}, {Carl}
#### Assignment: {11}

### Instructions
For each assignment use the provided template.  Use proper markdown and coding syntax.  

If you have note used Jupyter Notebooks before, please review: https://realpython.com/jupyter-notebook-introduction/

# Questions: 5.1, 5.2, 5.6

# Modify the code of the three query algorithms for the k-D tree so that it will report all the points tested during the search process. Do the algorithms ignore the nodes and points as they should logically?

(see code below for print() implimentation)

Response:

The query_kdtree() presents the points tested (and absence of nodes) in a straight forward way. From what I can observe, the presentation of the points 'travels down' the tree and stops at the appropriate leaf.

The range_query_orthagonial() presents the points tested in an unusual pattern (assuming I set-up the print() statements correctly). I observe skips in the print statements, skipping between depth=0 and depth=2 for example. 

The range_query_circular() presents all of the different depths of the tree, but there appears to be excluded prints() of all of the points and what depth they're, specifically, tested at. 

In [1]:
#Listing 5.1: Implementation of the k-D tree (kdtree1.py).

import sys
from point import *

class kDTreeNode():
    def __init__(self, point, left, right):
        self.point = point
        self.left = left
        self.right = right
    def __repr__(self):
        return str(self.point)

def kdcompare(r, p, depth):
    '''Returns the branch of searching on a k-d tree
    Input
        r: root
        p: point
        depth : starting depth of search
    Output
        A value of -1 (left branch), or 1 (right)'''
    k = len(p)
    dim = depth%k
    if p[dim] <= r.point[dim]:
        return -1
    else:
        return 1

def kdtree(points):
    '''
    Creates a k-d tree using a predefined order of points
    '''
    root = kDTreeNode(point=points[0], left=None, right=None)
    for p in points[1:]:
        node = kDTreeNode(point=p, left=None, right=None)
        p0, lr = query_kdtree(root, p, 0, False)
        if lr<0:
            p0.left = node
        else:
            p0.right = node
    return root

# always use the median point to split the data
def kdtree2(points, depth = 0):
    if len(points)==0:
        return
    k = len(points[0])
    axis = depth % k
    points.sort(key=lambda points: points[axis])
    pivot = len(points)//2
    while pivot<len(points)-1 and\
          points[pivot][axis]==points[pivot+1][axis]:
        pivot += 1
    return kDTreeNode(point=points[pivot],
                     left=kdtree2(points[:pivot],
                                 depth+1),
                     right=kdtree2(points[pivot+1:],
                                 depth+1))

def query_kdtree(t, p, depth=0, is_find_only=True):
    if t is None:
        print()
        return
    if t.point == p and is_find_only:
        print(t, p, depth)
        return t
    lr = kdcompare(t, p, depth)
    if lr<0:
        print(t, p, depth)
        child = t.left
    else:
        print(t, p, depth)
        child = t.right
    if child is None:
        if not is_find_only:
            print(t, p, depth)
            return t, lr
        else:
            print(t, p, depth)
            return
    return query_kdtree(child, p, depth+1, is_find_only)
    print(query_kdtree(child, p, depth+1, is_find_only))
    
if __name__ == '__main__':
    data1 = [ (2,2), (0,5), (8,0), (9,8),
            (7,14), (13,12), (14,13) ]
    points = [Point(d[0], d[1]) for d in data1]
    p = points[0]
    t1 = kdtree(points)
    t2 = kdtree2(points)
    
    print('---------------------------------------')
    print('kdtree:')
    print([ query_kdtree(t1, p) for p in points ])
    print('---------------------------------------')
    print('kdtree2:')
    print([ query_kdtree(t2, p) for p in points ])

(2, 2) (0, 5) 0
(2, 2) (0, 5) 0
(2, 2) (8, 0) 0
(2, 2) (8, 0) 0
(2, 2) (9, 8) 0
(8, 0) (9, 8) 1
(8, 0) (9, 8) 1
(2, 2) (7, 14) 0
(8, 0) (7, 14) 1
(9, 8) (7, 14) 2
(9, 8) (7, 14) 2
(2, 2) (13, 12) 0
(8, 0) (13, 12) 1
(9, 8) (13, 12) 2
(9, 8) (13, 12) 2
(2, 2) (14, 13) 0
(8, 0) (14, 13) 1
(9, 8) (14, 13) 2
(13, 12) (14, 13) 3
(13, 12) (14, 13) 3
---------------------------------------
kdtree:
(2, 2) (0, 5) 0
(0, 5) (0, 5) 1
(2, 2) (2, 2) 0
(2, 2) (7, 14) 0
(8, 0) (7, 14) 1
(9, 8) (7, 14) 2
(7, 14) (7, 14) 3
(2, 2) (8, 0) 0
(8, 0) (8, 0) 1
(2, 2) (9, 8) 0
(8, 0) (9, 8) 1
(9, 8) (9, 8) 2
(2, 2) (13, 12) 0
(8, 0) (13, 12) 1
(9, 8) (13, 12) 2
(13, 12) (13, 12) 3
(2, 2) (14, 13) 0
(8, 0) (14, 13) 1
(9, 8) (14, 13) 2
(13, 12) (14, 13) 3
(14, 13) (14, 13) 4
[(0, 5), (2, 2), (7, 14), (8, 0), (9, 8), (13, 12), (14, 13)]
---------------------------------------
kdtree2:
(8, 0) (0, 5) 0
(0, 5) (0, 5) 1
(8, 0) (2, 2) 0
(0, 5) (2, 2) 1
(2, 2) (2, 2) 2
(8, 0) (7, 14) 0
(0, 5) (7, 14) 1
(7, 14) (7, 14) 

In [2]:
#Listing 5.2: Orthogonal range querying the k-D tree (kdtree2a.py).

from kdtree1 import *

def range_query_orthogonal(t, rect, found, depth=0):
    if t is None:
        print(t, found, depth)
        return
    k = len(t.point)
    axis = depth%k
    if t.point[axis] < rect[axis][0]:
        range_query_orthogonal(t.right, rect, found, depth+1)
        print(t, found, depth)      #Carl's print
        return
    if t.point[axis] > rect[axis][1]:
        range_query_orthogonal(t.left, rect, found, depth+1)
        print(t, found, depth)      #Carl's print
        return
    x, y = t.point.x, t.point.y
    if not (rect[0][0]>x or rect[0][1]<x
            or
            rect[1][0]>y or rect[1][1]<y):
        found.append(t.point)
        print(t, found, depth)
    range_query_orthogonal(t.left, rect, found, depth+1)
    print(range_query_orthogonal(t.left, rect, found, depth+1))
    range_query_orthogonal(t.right, rect, found, depth+1)
    print(range_query_orthogonal(t.right, rect, found, depth+1))
    
def test():
    data1 = [ (2,2), (0,5), (8,0), (9,8), (7,14),
            (13,12), (14,13) ]
    points = [Point(d[0], d[1]) for d in data1]
    t1 = kdtree(points)
    rect = [ [1, 9], [2, 9] ]
    found = []
    range_query_orthogonal(t1, rect, found)
    
if __name__ == '__main__':
    test()

(2, 2) [Point(2, 2)] 0
None [Point(2, 2)] 2
None [Point(2, 2)] 2
None
None [Point(2, 2)] 2
None [Point(2, 2)] 2
None
None [Point(2, 2)] 2
None [Point(2, 2)] 2
None
None [Point(2, 2)] 2
None [Point(2, 2)] 2
None
None
(9, 8) [Point(2, 2), Point(9, 8)] 2
None [Point(2, 2), Point(9, 8)] 4
(7, 14) [Point(2, 2), Point(9, 8)] 3
None [Point(2, 2), Point(9, 8)] 4
(7, 14) [Point(2, 2), Point(9, 8)] 3
None
None [Point(2, 2), Point(9, 8)] 4
(13, 12) [Point(2, 2), Point(9, 8)] 3
None [Point(2, 2), Point(9, 8)] 4
(13, 12) [Point(2, 2), Point(9, 8)] 3
None
(8, 0) [Point(2, 2), Point(9, 8)] 1
(9, 8) [Point(2, 2), Point(9, 8), Point(9, 8)] 2
None [Point(2, 2), Point(9, 8), Point(9, 8)] 4
(7, 14) [Point(2, 2), Point(9, 8), Point(9, 8)] 3
None [Point(2, 2), Point(9, 8), Point(9, 8)] 4
(7, 14) [Point(2, 2), Point(9, 8), Point(9, 8)] 3
None
None [Point(2, 2), Point(9, 8), Point(9, 8)] 4
(13, 12) [Point(2, 2), Point(9, 8), Point(9, 8)] 3
None [Point(2, 2), Point(9, 8), Point(9, 8)] 4
(13, 12) [Point(2, 2), 

In [3]:
#Listing 5.3: Circular range querying the k-D tree (kdtree2b.py).

from kdtree1 import *

# range search for points within a radius of r around p
def range_query_circular(t, p, r, found, depth=0):
    if t is None:
        return
    if kdcompare(t, Point(p.x-r, p.y-r), depth)>0:
        range_query_circular(t.right, p, r, found, depth+1)
        print(t, p, r, found, depth)   #Carl's print
        return
    if kdcompare(t, Point(p.x+r, p.y+r), depth)<0:
        range_query_circular(t.left, p, r, found, depth+1)
        print(t, p, r, found, depth)   #Carl's print
        return
    if p.distance(t.point) <= r:
        found.append(t.point)
        print(t, p, r, found, depth)
    range_query_circular(t.left, p, r, found, depth+1)
    range_query_circular(t.right, p, r, found, depth+1)
    return

def test():
    data1 = [ (2,2), (0,5), (8,0), (9,8), (7,14),
            (13,12), (14,13) ]
    points = [Point(d[0], d[1]) for d in data1]
    p = Point(5,5)
    t1 = kdtree(points)
    found = []
    range_query_circular(t1, p, 5, found)
    print(found)
    
if __name__ == '__main__':
    test()

(2, 2) (5, 5) 5 [Point(2, 2)] 0
(0, 5) (5, 5) 5 [Point(2, 2), Point(0, 5)] 1
(9, 8) (5, 5) 5 [Point(2, 2), Point(0, 5), Point(9, 8)] 2
(7, 14) (5, 5) 5 [Point(2, 2), Point(0, 5), Point(9, 8)] 3
(13, 12) (5, 5) 5 [Point(2, 2), Point(0, 5), Point(9, 8)] 3
[Point(2, 2), Point(0, 5), Point(9, 8)]


# Test the performance of the range and nearest neighbor query methods for k-D trees. Listing 5.8 includes the necessary code that can be reused to create random point data sets for testing.

Note: I assume this question just means the range and nearest neighbor query methods in section 5.2 (listing 5.6 and 5.7).

In [4]:
# Listing 5.6: Circular range query using a PR k-D tree (prkdtree2.py).
#Carl is just making sure the code works in this cell, before passing the 'random' dataset through it (In [7])

from prkdtree1 import *
# range search for points within a radius of r around p
def range_query(t, p, r):
    def rquery(t, p, r, found, depth=0):
        if t is None:
            return
        w = t.xyrange[0][1] - t.xyrange[0][0]
        h = t.xyrange[1][1] - t.xyrange[1][0]
        if prkdcompare(t, Point(p.x-r-w, p.y-r-h),
                    depth)[1] > 0:
            rquery(t.right, p, r, found, depth+1)
            return
        if prkdcompare(t, Point(p.x+r+w, p.y+r+h),
                       depth)[1] < 0:
            rquery(t.left, p, r, found, depth+1)
            return
        if t.point is not None:
            if p.distance(t.point) <= r:
                found.append(t.point)
        rquery(t.left, p, r, found, depth+1)
        rquery(t.right, p, r, found, depth+1)
        return
    found = []
    if t is not None:
        rquery(t, p, r, found)
    return found

if __name__ == '__main__':
    data1 = [ (2,2), (0,5), (8,0), (9,8), (7,14),
    (13,12), (14,13) ]
    points = [Point(d[0], d[1]) for d in data1]
    px = [p.x for p in points]
    py = [p.y for p in points]
    xmin = min(px)
    xmax = max(px)
    ymin = min(py)
    ymax = max(py)
    xylim = [ [xmin, xmax], [ymin, ymax] ]
    t = prkdtree(points, xylim)
    p = Point(5,5)
    print(range_query(t, p, 5))

[Point(2, 2), Point(0, 5), Point(9, 8)]


In [5]:
#Listing 5.7: Nearest neighbor query using a PR k-D tree (prkdtree3.py)
#Carl is just making sure the code works in this cell, before passing the 'random' dataset through it (In [8])

from prkdtree1 import *
from kdtree3 import * # use update_neighbors and INF

prmaxdist = INF

def prkdtree_nnquery(t, p, n, found, depth=0):
    global prmaxdist
    if t is None:
        return
    if t.is_leaf() and t.point is not None:
        prmaxdist = update_neighbors(t.point, p, found, n)
        return
    axis,dir = prkdcompare(t, p, depth)
    if dir<0:
        nearer_tree, farther_tree = t.left, t.right
    else:
        nearer_tree, farther_tree = t.right, t.left
    prkdtree_nnquery(nearer_tree, p, n, found, depth+1)
    #prmaxdist = update_neighbors(t.point, p, found, n)
    if (t.center-p[axis]) < prmaxdist:
        prkdtree_nnquery(farther_tree, p, n, found, depth+1)
    return

def nearest_neighbor_query(t, p, n=1):
    nearest_neighbors = []
    prkdtree_nnquery(t, p, n, nearest_neighbors)
    return nearest_neighbors[:n]

if __name__ == '__main__':
    data1 = [ (2,2), (0,5), (8,0), (9,8), (7,14),
    (13,12), (14,13) ]
    points = [Point(d[0], d[1]) for d in data1]
    px = [p.x for p in points]
    py = [p.y for p in points]
    xmin = min(px)
    xmax = max(px)
    ymin = min(py)
    ymax = max(py)
    xylim = [ [xmin, xmax], [ymin, ymax] ]
    t = prkdtree(points, xylim)
    n = 3
    p = Point(5,5)
    nearests = nearest_neighbor_query(t, p, n)
    print ([x[0] for x in nearests[:n]])
    print ([x[1] for x in nearests[:n]])
    print (sorted([p.distance(x) for x in points])[:n])

[Point(2, 2), Point(0, 5), Point(9, 8)]
[4.242640687119285, 5.0, 5.0]
[4.242640687119285, 5.0, 5.0]


In [6]:
#Listing 5.8: Performance tests of k-D trees (kdtree_performance.py).
#this cell Carl generated the 'random' data..

from kdtree1 import *
from prkdtree1 import *

import random
import time
import sys
import string

if __name__ == '__main__':
    npts = 1000 # default number of points
    if len(sys.argv)==2: # user specified number of points
        if sys.argv[1].isdigit() is True:
            npts = string.atoi(sys.argv[1])
    points = []
    for i in range(npts):
        p = Point(random.random(), random.random())
        points.append(p)

    time1 = time.time()
    kdt1 = kdtree(points)
    time2 = time.time()
    treet1 = time2-time1
    
    time1 = time.time()
    kdt2 = kdtree2(points)
    time2 = time.time()
    treet2 = time2-time1
    
    px = [p.x for p in points]
    py = [p.y for p in points]
    xmin = min(px)
    xmax = max(px)
    ymin = min(py)
    ymax = max(py)
    xylim = [ [xmin, xmax], [ymin, ymax] ]
    time1 = time.time()
    kdt3 = prkdtree(points, xylim)
    time2 = time.time()
    treet3 = time2-time1
    
    print (npts, "|", treet1, treet2, treet3, "|")
    
    n = 100
    t1 = 0 # time finding 1000 points in kdtree
    t2 = 0 # time for balanced kdtree
    t3 = 0 # time for pr kd tree
    t4 = 0 # time for linear search
    pp = random.sample(points, n)
    for p in pp:
        time1 = time.time()
        p1 = query_kdtree(kdt1, p)
        time2 = time.time()
        t1 = t1 + time2-time1
        time1 = time.time()
        p1 = query_kdtree(kdt2, p)
        time2 = time.time()
        t2 = t2 + time2-time1
        time1 = time.time()
        p1 = query_prkdtree(kdt3, p)
        time2 = time.time()
        t3 = t3 + time2-time1
        time1 = time.time()
        for i in range(len(points)):
            if p == points[i]:
                break
        time2 = time.time()
        t4 = t4 + time2-time1
        
    print (t1, t2, t3, t4)

1000 | 0.009974956512451172 0.00299072265625 0.010970354080200195 |
0.000997304916381836 0.000997781753540039 0.0019953250885009766 0.011967182159423828


In [7]:
#testing range_query with random generator data

# Listing 5.6: Circular range query using a PR k-D tree (prkdtree2.py).

from prkdtree1 import *
# range search for points within a radius of r around p
def range_query(t, p, r):
    def rquery(t, p, r, found, depth=0):
        if t is None:
            return
        w = t.xyrange[0][1] - t.xyrange[0][0]
        h = t.xyrange[1][1] - t.xyrange[1][0]
        if prkdcompare(t, Point(p.x-r-w, p.y-r-h),
                    depth)[1] > 0:
            rquery(t.right, p, r, found, depth+1)
            return
        if prkdcompare(t, Point(p.x+r+w, p.y+r+h),
                       depth)[1] < 0:
            rquery(t.left, p, r, found, depth+1)
            return
        if t.point is not None:
            if p.distance(t.point) <= r:
                found.append(t.point)
        rquery(t.left, p, r, found, depth+1)
        rquery(t.right, p, r, found, depth+1)
        return
    found = []
    if t is not None:
        rquery(t, p, r, found)
    return found

if __name__ == '__main__': 
    # 'points' is alreayd stored in memory
    t1 = time.time()
    px = [p.x for p in points]
    py = [p.y for p in points]
    xmin = min(px)
    xmax = max(px)
    ymin = min(py)
    ymax = max(py)
    xylim = [ [xmin, xmax], [ymin, ymax] ]
    t = prkdtree(points, xylim)
    p = Point(0,0)
    range_query(t, p, 5)
    t2 = time.time()
    print('Range query took', t2-t1, 'seconds!')

Range query took 0.016954898834228516 seconds!


In [8]:
#testing nearest neighbors with random generator data

from prkdtree1 import *
from kdtree3 import * # use update_neighbors and INF

prmaxdist = INF

def prkdtree_nnquery(t, p, n, found, depth=0):
    global prmaxdist
    if t is None:
        return
    if t.is_leaf() and t.point is not None:
        prmaxdist = update_neighbors(t.point, p, found, n)
        return
    axis,dir = prkdcompare(t, p, depth)
    if dir<0:
        nearer_tree, farther_tree = t.left, t.right
    else:
        nearer_tree, farther_tree = t.right, t.left
    prkdtree_nnquery(nearer_tree, p, n, found, depth+1)
    #prmaxdist = update_neighbors(t.point, p, found, n)
    if (t.center-p[axis]) < prmaxdist:
        prkdtree_nnquery(farther_tree, p, n, found, depth+1)
    return

def nearest_neighbor_query(t, p, n=1):
    nearest_neighbors = []
    prkdtree_nnquery(t, p, n, nearest_neighbors)
    return nearest_neighbors[:n]

if __name__ == '__main__':
    # 'points' is already stored in memory
    t1 = time.time()
    px = [p.x for p in points]
    py = [p.y for p in points]
    xmin = min(px)
    xmax = max(px)
    ymin = min(py)
    ymax = max(py)
    xylim = [ [xmin, xmax], [ymin, ymax] ]
    t = prkdtree(points, xylim)
    n = 3
    p = Point(0,0)
    nearests = nearest_neighbor_query(t, p, n)
    [x[0] for x in nearests[:n]]
    [x[1] for x in nearests[:n]]
    sorted([p.distance(x) for x in points])[:n]
    t2 = time.time()
    t3 = t2-t1
    print('Nearest Neighbors query (n=3) took', t3, 'seconds!')

Nearest Neighbors query (n=3) took 0.009973287582397461 seconds!


# What happens if distance (or spatial information in general) is not the only criterion needed for nearest neighbor query? For example, if each point is associated with a total population and we require the nearest neighbors to have more than 100 people. Modify the code to return the correct results.

In [9]:
#testing nearest neighbors with random generator data, with change to 'n'

from prkdtree1 import *
from kdtree3 import * # use update_neighbors and INF

prmaxdist = INF

def prkdtree_nnquery(t, p, n, found, depth=0):
    global prmaxdist
    if t is None:
        return
    if t.is_leaf() and t.point is not None:
        prmaxdist = update_neighbors(t.point, p, found, n)
        return
    axis,dir = prkdcompare(t, p, depth)
    if dir<0:
        nearer_tree, farther_tree = t.left, t.right
    else:
        nearer_tree, farther_tree = t.right, t.left
    prkdtree_nnquery(nearer_tree, p, n, found, depth+1)
    #prmaxdist = update_neighbors(t.point, p, found, n)
    if (t.center-p[axis]) < prmaxdist:
        prkdtree_nnquery(farther_tree, p, n, found, depth+1)
    return

def nearest_neighbor_query(t, p, n=1):
    nearest_neighbors = []
    prkdtree_nnquery(t, p, n, nearest_neighbors)
    return nearest_neighbors[:n]

if __name__ == '__main__':
    # 'points' is already stored in memory
    t1 = time.time()
    px = [p.x for p in points]
    py = [p.y for p in points]
    xmin = min(px)
    xmax = max(px)
    ymin = min(py)
    ymax = max(py)
    xylim = [ [xmin, xmax], [ymin, ymax] ]
    t = prkdtree(points, xylim)
    n = 100                                                           #Carl changed the n from 3 to 100
    p = Point(0,0)
    nearests = nearest_neighbor_query(t, p, n)
    [x[0] for x in nearests[:n]]
    [x[1] for x in nearests[:n]]
    sorted([p.distance(x) for x in points])[:n]
    t2 = time.time()
    t4 = t2-t1
    print('Nearest Neighbors query (n=100) took', t4, 'seconds!')

Nearest Neighbors query (n=100) took 0.010970592498779297 seconds!


Observation:

After changing the n-requirement from 3 to 100, it's observed that there's an increase in execution time. 

In [10]:
print('n=100 took', t4-t3, 'seconds longer than n=3 to execute!')

n=100 took 0.000997304916381836 seconds longer than n=3 to execute!


End of notebook. -CJB 11/17/19